In [1]:
import pandas as pd
import numpy as np

# Preprocessing Sources

We will use data from the [Center for Systems Science and Engineering (CSSE) at Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19) repo at GitHub.

## Deaths dataset

In [2]:
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [3]:
deaths.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2745,2751,2762,2772,2782,2792,2802,2812,2836,2855
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2435,2436,2438,2440,2441,2442,2444,2445,2447,2447
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3381,3388,3395,3401,3405,3411,3418,3426,3433,3440
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,127,127,127,127,127,127,127,127,127,127
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,677,685,696,704,709,715,725,731,735,742


Dropping unnecessary features as we'll work with country-wise information.

In [4]:
deaths.drop(columns=['Province/State'], inplace=True)

### Changing data display from wide format to long format

This will be useful later on for easier data viz.

It's simply a reshape of the dataframe grabbing all the dates columns and turning them into one `Date` column and using each of their values for a new column called `Deaths`.

In [5]:
dates_list = deaths.columns.drop(['Country/Region', 'Lat', 'Long' ])

deaths_df = deaths.melt(id_vars=['Country/Region', 'Lat', 'Long' ], value_vars=dates_list,
           var_name='Date', value_name='Deaths')

deaths_df.head()

,Country/Region,Lat,Long,Date,Deaths
0,Afghanistan,33.93911,67.709953,1/22/20,0
1,Albania,41.15330,20.168300,1/22/20,0
2,Algeria,28.03390,1.659600,1/22/20,0
3,Andorra,42.50630,1.521800,1/22/20,0
4,Angola,-11.20270,17.873900,1/22/20,0


As some countries have information per `Province/State` we need to group information by `Country/Region` and `Date` to get the correspondent total `Deaths`.

In [6]:
deaths_grouped = deaths_df.groupby(by=['Country/Region', 'Date'], as_index=False).agg({
    'Lat': np.mean, # We'll use the mean to get a correct location to be used in Tableau
    'Long': np.mean,
    'Deaths': sum
})

## Confirmed cases data set

Given that's the same format as in the `deaths` dataset, we'll perform the same operations.

In [7]:
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,63598,63819,64122,64575,65080,65486,65728,66275,66903,67743
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132032,132071,132095,132118,132153,132176,132209,132215,132229,132244
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,125485,125693,125896,126156,126434,126651,126860,127107,127361,127646
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,13555,13569,13569,13569,13569,13569,13569,13569,13664,13671
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,30787,31045,31438,31661,31909,32149,32441,32623,32933,33338


In [8]:
confirmed.drop(columns=['Province/State'], inplace=True)

confirmed_df = confirmed.melt(id_vars=['Country/Region', 'Lat', 'Long' ], value_vars=dates_list,
           var_name='Date', value_name='Confirmed')


confirmed_grouped = confirmed_df.groupby(by=['Country/Region', 'Date'], as_index=False).agg({
    'Lat': np.mean, # We'll use the mean to get a correct location to be used in Tableau
    'Long': np.mean,
    'Confirmed': sum
})

## Recovered cases data set

In [9]:
recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [10]:
recovered.drop(columns=['Province/State'], inplace=True)

recovered_df = recovered.melt(id_vars=['Country/Region', 'Lat', 'Long' ], value_vars=dates_list,
           var_name='Date', value_name='Recovered')


recovered_grouped = recovered_df.groupby(by=['Country/Region', 'Date'], as_index=False).agg({
    'Lat': np.mean, # We'll use the mean to get a correct location to be used in Tableau
    'Long': np.mean,
    'Recovered': sum
})

# Merging data

Checking dataset's number of rows to match

In [11]:
recovered_grouped.shape, deaths_grouped.shape, confirmed_grouped.shape 

((94272, 5), (94272, 5), (94272, 5))

`recovered_df` dataset has a different layout from the others as some countries don't have the data split in `Province/State`. 

See the example with *Canada* :

In [12]:
recovered_df[(recovered_df['Country/Region'] == 'Canada')].head()

,Country/Region,Lat,Long,Date,Recovered
39,Canada,56.1304,-106.3468,1/22/20,0
299,Canada,56.1304,-106.3468,1/23/20,0
559,Canada,56.1304,-106.3468,1/24/20,0
819,Canada,56.1304,-106.3468,1/25/20,0
1079,Canada,56.1304,-106.3468,1/26/20,0


In [13]:
confirmed_df[(confirmed_df['Country/Region'] == 'Canada')].head()

,Country/Region,Lat,Long,Date,Confirmed
39,Canada,53.9333,-116.5765,1/22/20,0
40,Canada,53.7267,-127.6476,1/22/20,0
41,Canada,0.0000,0.0000,1/22/20,0
42,Canada,0.0000,0.0000,1/22/20,0
43,Canada,53.7609,-98.8139,1/22/20,0


In [14]:
deaths_df[(deaths_df['Country/Region'] == 'Canada') ].head()

,Country/Region,Lat,Long,Date,Deaths
39,Canada,53.9333,-116.5765,1/22/20,0
40,Canada,53.7267,-127.6476,1/22/20,0
41,Canada,0.0000,0.0000,1/22/20,0
42,Canada,0.0000,0.0000,1/22/20,0
43,Canada,53.7609,-98.8139,1/22/20,0


To avoid losing data while merging, we'll omit using `Lat` and `Long` from `deaths_grouped` and `confirmed_grouped` as it could lead to a conflict because it won't match the ones at `recovered_grouped`.

In [15]:
data = recovered_grouped.merge(
    deaths_grouped[['Country/Region', 'Date', 'Deaths']], on=['Country/Region', 'Date']).merge(
    confirmed_grouped[['Country/Region', 'Date', 'Confirmed']], on=['Country/Region', 'Date'])

In [16]:
data.sample(5)

,Country/Region,Date,Lat,Long,Recovered,Deaths,Confirmed
49479,Libya,6/19/20,26.335100,17.228331,83,10,520
94200,Zimbabwe,7/28/20,-19.015438,29.154857,604,40,2817
24550,Dominica,1/1/21,15.415000,-61.371000,83,0,88
33730,Greece,5/23/20,39.074200,21.824300,1374,171,2876
49397,Libya,4/3/21,26.335100,17.228331,148288,2684,161088


In [17]:
data['Country/Region'].nunique()

192

# Combining our data with total population for each country

We get another dataset containing the population for each country in order to be able to present data normalized by inhabitants.

Source: [World Population Review](https://worldpopulationreview.com/countries)

In [18]:
population = pd.read_csv('population.csv').drop(columns='Rank')

population['pop2021'] = population['pop2021'] * 1000
population['pop2020'] = population['pop2020'] * 1000

In [19]:
population.head()

,name,pop2021,pop2020,GrowthRate,area,Density
0,China,1.444216e+09,1.439324e+09,1.0034,9706961,147.7068
1,India,1.393409e+09,1.380004e+09,1.0097,3287590,415.6290
2,United States,3.329151e+08,3.310027e+08,1.0058,9372610,35.1092
3,Indonesia,2.763618e+08,2.735236e+08,1.0104,1904569,142.0928
4,Pakistan,2.251999e+08,2.208923e+08,1.0195,881912,245.5634


We'll check if every country's name is spelled the same in both datasets.

In [20]:
 [country for country in data['Country/Region'].unique() if country not in population.name.unique()]

['Burma',
 'Cabo Verde',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 "Cote d'Ivoire",
 'Czechia',
 'Diamond Princess',
 'Eswatini',
 'Holy See',
 'Korea, South',
 'Kosovo',
 'MS Zaandam',
 'Taiwan*',
 'US',
 'West Bank and Gaza']

We'll need to manually input the names in our source dataset in order to match the population one.

In [21]:
# We'll get rid of the Diamond Princess and MS Zaandam as we'll only consider countries.
diamond = data[data['Country/Region'].str.contains('Diamond Princess') == True].index
data.drop(diamond, inplace=True)
zaandam = data[data['Country/Region'].str.contains('MS Zaandam') == True].index
data.drop(zaandam, inplace=True)
# We'll drop territories under dispute as they may be duplicating data.
kosovo = data[data['Country/Region'].str.contains('Kosovo') == True].index
data.drop(kosovo, inplace=True)
west_bank = data[data['Country/Region'].str.contains('West Bank and Gaza') == True].index
data.drop(west_bank, inplace=True)

In [22]:
data['Country/Region'] = data['Country/Region'].str.replace('Burma', 'Myanmar')
data['Country/Region'] = data['Country/Region'].str.replace('Cabo Verde', 'Cape Verde')
data['Country/Region'] = data['Country/Region'].str.replace('''Congo (Brazzaville)''', 'Republic of the Congo',
                                                            regex=False)
data['Country/Region'] = data['Country/Region'].str.replace('''Congo (Kinshasa)''', 'DR Congo',
                                                            regex=False)
data['Country/Region'] = data['Country/Region'].str.replace('''Cote d'Ivoire''', 'Ivory Coast')
data['Country/Region'] = data['Country/Region'].str.replace('Czechia', 'Czech Republic')
data['Country/Region'] = data['Country/Region'].str.replace('Eswatini', 'Swaziland')
data['Country/Region'] = data['Country/Region'].str.replace('Holy See', 'Vatican City')
data['Country/Region'] = data['Country/Region'].str.replace('Korea, South', 'South Korea')
data['Country/Region'] = data['Country/Region'].str.replace('Taiwan*', 'Taiwan', regex=False)
data['Country/Region'] = data['Country/Region'].str.replace('US', 'United States')


Test to check that every country is accounted for at the population dataframe:

In [23]:
missing_countries = [country for country in data['Country/Region'].unique() 
                      if country not in population.name.unique()]
def test_countries():
    assert len(missing_countries) == 0, 'There are some countries not listed in population dataframe'
    return '✅ Everything OK'

test_countries()

'✅ Everything OK'

# Export data

In [24]:
data.to_csv('covid.csv', index=False)